# OTC Markets
We wraped the data from [OTC Markets.](https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume) Then, we were fail to wrap all of data (18524 companies) by using splinter "click_link" because the MORE button design of this website is a Javescript which would be triggered when user click it. Therefore, we decided to click MORE button by ourselves to show all of data and wrap it.

In [8]:
from splinter import Browser
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [9]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.otcmarkets.com/market-activity/current-market/ALL/active/dollarVolume"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [60]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [61]:
#We click MORE button for several times to get all of data. Then, wrap the whole table content.
OTC_market_soup = soup.find('table', class_='_1DoSj9nrVo')
#Change bs4 object into html format
html_content = OTC_market_soup.prettify()

In [62]:
#Using Pandas to read html
OTC_market_list = pd.read_html(html_content)

In [64]:
#Convert it to DataFrame and rename columns name
OTC_market_table = OTC_market_list[0]
OTC_market_table.rename(columns={"Symbol  Sym":"Company","% Change  % Change": "Change_Percent", "$ Vol  $ Vol": "Vol", "Share Vol  Share Vol":"Share Vol",
                  "Trades  Trades":"Trades"},inplace=True)
OTC_market_table.head(10)


,Company,Price Price,Change_Percent,Vol,Share Vol,Trades
0,TSCDY,9.8950,1.59,91637772,9263861,220
1,TCEHY,49.2400,-0.93,65130773,1320502,4242
2,NSRGY,96.5300,0.77,53018306,550680,1974
3,CSGKF,13.2800,-1.56,41993141,3151688,559
4,RHHBY,33.0600,1.26,41937727,1244087,1685
5,NPSNY,51.4500,-0.17,28886313,561582,976
6,SAPGF,127.8500,-0.62,23820273,186421,12
7,ANPDY,174.5000,-1.61,22454712,128054,56
8,LUKOY,85.5000,-0.94,21700288,254936,261
9,OGRMF,7.4500,2.96,19558126,2609580,5444


In [68]:
#Wrap the icon image link from website in order to identify which Symbol it is.
#Find all of icon image link
icon_list = OTC_market_soup.find_all("div",class_="_2_IqcDGekT")
symbol = []
for x in icon_list:
    #Only take symbol portion from link
    symbol.append(x.img["src"][12:14])

In [70]:
#Combine symbol data into OTC Market
OTC_market_table["Symbol"] = symbol
OTC_market_table.head()

,Company,Price Price,Change_Percent,Vol,Share Vol,Trades,Symbol
0,TSCDY,9.895,1.59,91637772,9263861,220,ps
1,TCEHY,49.240,-0.93,65130773,1320502,4242,ps
2,NSRGY,96.530,0.77,53018306,550680,1974,ps
3,CSGKF,13.280,-1.56,41993141,3151688,559,ps
4,RHHBY,33.060,1.26,41937727,1244087,1685,qx


We only chose QX and QB symbol.

In [102]:
OTC_QX_QB = OTC_market_table.loc[(OTC_market_table.Symbol == "qx") | (OTC_market_table.Symbol == "qb"),:]
OTC_QX_QB.reset_index(drop=True,inplace=True)
OTC_QX_QB.head(10)

,Company,Price Price,Change_Percent,Vol,Share Vol,Trades,Symbol
0,RHHBY,33.06,1.26,41937727,1244087,1685,qx
1,OGRMF,7.45,2.96,19558126,2609580,5444,qx
2,GBTC,6.61,6.44,16190892,2461578,3063,qx
3,FNMAS,11.08,-0.36,10448862,946815,984,qb
4,GBOOY,31.54,-3.04,7432845,235751,108,qx
5,FMCKJ,10.93,-0.36,6175952,567823,215,qb
6,ADDYY,128.54,0.71,6149869,32348,422,qx
7,PUBGY,14.86,-1.07,5783314,387785,153,qx
8,FERGY,7.05,1.00,5586375,793699,315,qx
9,CURLF,9.91,0.59,5441317,559688,1629,qx


In [103]:
OTC_QX_QB.to_csv("part_qx_qb.csv")